# Industry Momentum

In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import DataReader as pdr
import plotly.graph_objects as go
import statsmodels.api as sm

# Read data and clean-up missing data (coded -99.99)
ff48 = pdr("48_Industry_Portfolios", "famafrench", start=1900)[0]

# Clean-up missings
for c in ff48.columns:
    ff48[c] = np.where(ff48[c]==-99.99, np.nan, ff48[c])
ff48 = ff48/100

# Estimation window in months for past average returns
WINDOW = 12

asset_list = ff48.columns
dates = ff48.index[WINDOW:]

In [2]:
# Fill missings with cross-sectional average
for c in ff48.columns:
    ff48[c]=ff48[c].fillna(ff48.mean(axis=1))
# ff48.to_csv('ff48.csv')


In [3]:
# Calculate rolling average returns
avgs = ff48.rolling(WINDOW).mean()  # this average is inclusive of the return in a given row
avgs = avgs.iloc[WINDOW:]
avgs

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1927-07,0.016858,0.020725,0.018057,0.049908,0.033725,0.005625,0.000150,0.027667,0.014592,0.015567,...,0.005958,0.022383,-0.038567,0.017408,0.019675,-0.012692,0.035767,0.014675,0.012633,0.003725
1927-08,0.016658,0.020492,0.016639,0.024067,0.031500,-0.002233,0.005817,-0.007600,0.018008,0.018958,...,0.011075,0.016958,-0.045208,0.024433,0.020242,-0.018350,0.033717,0.011933,0.014925,0.003025
1927-09,0.023242,0.023983,0.020811,0.026558,0.034308,-0.008858,0.004400,-0.002558,0.019508,0.023258,...,0.018500,0.020483,-0.033983,0.030567,0.025658,-0.013100,0.039917,0.014300,0.024300,0.006600
1927-10,0.018792,0.025592,0.021325,0.028650,0.033158,-0.008750,0.009033,-0.009817,0.021875,0.031200,...,0.022850,0.018825,-0.039317,0.028208,0.027625,-0.001075,0.052442,0.015550,0.024642,0.010658
1927-11,0.018592,0.026467,0.024686,0.031150,0.034942,-0.011583,0.011283,0.002408,0.024600,0.031358,...,0.033708,0.020733,-0.044292,0.032867,0.026525,0.011150,0.055367,0.021775,0.026583,0.016650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05,0.026133,0.002850,0.012100,0.006242,0.014400,-0.056058,-0.041783,-0.015617,-0.001208,-0.015542,...,-0.003208,-0.014475,0.007000,-0.013425,-0.006225,-0.008075,0.010142,-0.009442,-0.002858,0.001658
2022-06,0.017717,0.003808,0.013692,0.007042,0.003050,-0.076867,-0.052142,-0.025733,-0.003125,-0.032892,...,-0.007250,-0.016783,0.003158,-0.023617,-0.013100,-0.015983,0.011250,-0.021125,-0.012375,-0.005850
2022-07,0.025583,0.008800,0.012633,0.010717,0.002717,-0.069375,-0.031717,-0.013533,-0.005792,-0.027067,...,-0.000158,-0.004833,0.008742,-0.009342,-0.006942,-0.008600,0.013800,-0.013067,-0.005108,0.002375


In [4]:
ports = avgs.apply(lambda x: pd.qcut(x, 5,labels=False), axis=1)
ports
# avgs.apply(lambda x: pd.qcut(x, 5,labels=False), axis=1)

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1927-07,1,3,2,4,4,0,0,3,1,1,...,0,3,0,2,3,0,4,1,1,0
1927-08,2,3,1,3,4,0,0,0,2,2,...,0,2,0,3,3,0,4,1,1,0
1927-09,2,2,1,3,4,0,0,0,1,2,...,1,1,0,3,3,0,4,1,3,0
1927-10,1,3,1,3,4,0,0,0,2,3,...,2,1,0,3,3,0,4,1,2,0
1927-11,1,2,1,3,4,0,0,0,1,3,...,3,1,0,3,2,0,4,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05,4,3,4,3,4,0,0,0,2,0,...,2,0,3,1,2,1,3,1,2,3
2022-06,4,3,4,4,3,0,0,0,3,0,...,2,1,3,0,2,1,4,0,2,2
2022-07,4,3,4,4,3,0,0,0,2,0,...,2,2,3,1,1,1,4,1,2,3


In [5]:
# Portfolio of the most recent winners
hi = pd.DataFrame(dtype=float, columns=ports.columns, index=ports.index)
for c in ports.columns:
    hi[c] = (ports[c]==4)*1.0
hi=hi.div(hi.sum(axis=1), axis=0)  # Normalize so weights sum to 1
hi

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1927-07,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
1927-08,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
1927-09,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
1927-10,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
1927-11,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05,0.1,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-06,0.1,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
2022-07,0.1,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0


In [6]:
# Portfolio of the most recent losers
lo = pd.DataFrame(dtype=float, columns=ports.columns, index=ports.index)
for c in ports.columns:
    lo[c] = (ports[c]==0)*1.0
lo=lo.div(lo.sum(axis=1), axis=0)  # Normalize so weights sum to 1
lo

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1927-07,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,...,0.1,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.1
1927-08,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.1,0.0,0.0,...,0.1,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.1
1927-09,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.1,0.0,0.0,...,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.1
1927-10,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.1,0.0,0.0,...,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.1
1927-11,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.1,0.0,0.0,...,0.0,0.0,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.1,0.0,0.1,...,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-06,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.1,0.0,0.1,...,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0
2022-07,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.1,0.0,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#  Multiply weights by returns in next month
rets = pd.DataFrame(dtype=float,columns=['hi','lo'],index=hi.index[1:])
for d in rets.index:
    rets.loc[d,'hi'] = hi.loc[d-1] @ ff48.loc[d]
    rets.loc[d,'lo'] = lo.loc[d-1] @ ff48.loc[d]
rets.index = rets.index.to_timestamp('M')
rets

,hi,lo
Date,,
1927-08-31,0.02605,0.04188
1927-09-30,0.06663,0.03243
1927-10-31,-0.02869,-0.02931
1927-11-30,0.07888,0.06609
1927-12-31,0.07808,0.02371
...,...,...
2022-05-31,0.00300,0.01138
2022-06-30,-0.10438,-0.10373
2022-07-31,0.07177,0.12674


In [12]:
# Plot cumulative returns
trace_hi  = go.Scatter(x=rets.index, y=(1+rets.hi).cumprod(), mode="lines", name='Winners')
trace_lo= go.Scatter(x=rets.index, y=(1+rets.lo).cumprod(), mode="lines", name='Losers')
fig = go.Figure()
fig.add_trace(trace_hi)
fig.add_trace(trace_lo)
fig.update_yaxes(type="log")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.show()

In [13]:
# Plot cumulative returns - nicer formatting
string =  "Strategy: Buy Past Winners<br>"
string += "Date: %{x}<br>"
string += "FV of $1: $%{y:,.2f}<br>"
string += "<extra></extra>"
trace_hi  = go.Scatter(x=rets.index, y=(1+rets.hi).cumprod(), mode="lines", name='Buy Winners', hovertemplate=string)
string =  "Strategy: Buy Past Losers<br>"
string += "Date: %{x}<br>"
string += "FV of $1: $%{y:,.2f}<br>"
string += "<extra></extra>"
trace_lo= go.Scatter(x=rets.index, y=(1+rets.lo).cumprod(), mode="lines", name='Sell Losers',hovertemplate=string)
fig = go.Figure()
fig.add_trace(trace_hi)
fig.add_trace(trace_lo)
fig.update_yaxes(type="log", title="FV of $1")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.show()

In [14]:
# Plot cumulative returns of long-short strategy
string =  "Strategy: Buy Winners and Short Losers<br>"
string += "Date: %{x}<br>"
string += "FV of $1: $%{y:,.2f}<br>"
string += "<extra></extra>"
trace  = go.Scatter(x=rets.index, y=(1+rets.hi-rets.lo).cumprod(), mode="lines", name='Long-Short', hovertemplate=string)
fig = go.Figure()
fig.add_trace(trace)
fig.update_yaxes(type="log",title='FV of $1')
fig.update_layout(title='Industry Momentum')
fig.show()

### Performance Statistics

In [ ]:
# Portfolio alpha and beta
ff3 = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[0]/100
rets.index=rets.index.to_period("M")
df = rets.join(ff3[['Mkt-RF','RF']])
df['hi-rf'] = df['hi']-df['RF']
df['lo-rf'] = df['lo']-df['RF']
df['hi-lo'] = df['hi']-df['lo']

In [18]:
def mkt_model(varname):
    mm = sm.OLS(df[varname], sm.add_constant(df['Mkt-RF'])).fit()
    print(mm.summary())

In [19]:
mkt_model('hi-rf')

                            OLS Regression Results                            
Dep. Variable:                  hi-rf   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.810
Method:                 Least Squares   F-statistic:                     4863.
Date:                Thu, 27 Oct 2022   Prob (F-statistic):               0.00
Time:                        13:54:43   Log-Likelihood:                 2519.9
No. Observations:                1142   AIC:                            -5036.
Df Residuals:                    1140   BIC:                            -5026.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0049      0.001      6.166      0.0

In [23]:
# These are monthly returns
alpha,beta = sm.OLS(df['hi-rf'], sm.add_constant(df['Mkt-RF'])).fit().params
print(f'Annualized alpha of long portfolio: {alpha*12: .2%}')

Annualized alpha of long portfolio:  5.88%


In [20]:
mkt_model('lo-rf')

                            OLS Regression Results                            
Dep. Variable:                  lo-rf   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.810
Method:                 Least Squares   F-statistic:                     4855.
Date:                Thu, 27 Oct 2022   Prob (F-statistic):               0.00
Time:                        13:55:02   Log-Likelihood:                 2356.3
No. Observations:                1142   AIC:                            -4709.
Df Residuals:                    1140   BIC:                            -4698.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0037      0.001     -4.065      0.0

In [24]:
# These are monthly returns
alpha,beta = sm.OLS(df['lo-rf'], sm.add_constant(df['Mkt-RF'])).fit().params
print(f'Annualized alpha of long portfolio: {alpha*12: .2%}')

Annualized alpha of long portfolio: -4.47%


In [21]:
mkt_model('hi-lo')

                            OLS Regression Results                            
Dep. Variable:                  hi-lo   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     40.88
Date:                Thu, 27 Oct 2022   Prob (F-statistic):           2.35e-10
Time:                        13:55:09   Log-Likelihood:                 1934.7
No. Observations:                1142   AIC:                            -3865.
Df Residuals:                    1140   BIC:                            -3855.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0086      0.001      6.504      0.0

In [22]:
# These are monthly returns
alpha,beta = sm.OLS(df['hi-lo'], sm.add_constant(df['Mkt-RF'])).fit().params
print(f'Annualized alpha of long-short portfolio: {alpha*12: .2%}')

Annualized alpha of long-short portfolio:  10.35%
